> **Dokumentacja Użytkownika**
> *Wersja oprogramowania: 4.0 (FEM Integrated)*
> *Data aktualizacji: Bieżąca*

# 📖 Instrukcja Obsługi: Optymalizator Słupa v4.0

## 1. Wstęp i Przeznaczenie Systemu
Aplikacja służy do projektowania i optymalizacji asymetrycznych słupów złożonych (ceownik + płaskownik). System łączy szybkość obliczeń analitycznych (zgodnych z Eurokodem 3 i Teorią Własowa) z dokładnością weryfikacji numerycznej (FEM/MES) opartej o solver CalculiX.

### Główny Przepływ Pracy (Workflow)
Poniższy schemat ilustruje zalecaną ścieżkę pracy z programem:

```mermaid
graph TD
    A[Start: Dashboard] -->|Definicja Obciążeń| B(Obliczenia Analityczne);
    B --> C[Selektor Wyników];
    C -->|Filtracja i Wybór| D{Wybór Kandydatów};
    D -->|Przekazanie| E[Analiza FEM];
    E -->|Solver CalculiX| F[Post-Processing];
    F --> G{Decyzja};
    G -->|Zgodność| H[Raport Końcowy];
    G -->|Rozbieżność| I[Korekta Modelu / Gęstsza Siatka];
    I --> E;
```

## 2. KROK 1: Definicja Problemu (Zakładka 1: Dashboard)

To centrum sterowania. Tutaj definiujesz geometrię, obciążenia i zakres poszukiwań.

### A. Panel Obciążeń (Lewa strona)
Parametry te definiują siły działające na słup. Pamiętaj o spójności jednostek (Newtony, milimetry).

| Parametr | Jednostka | Opis Fizyczny | Wpływ na wynik |
| :--- | :--- | :--- | :--- |
| **Fx** | $[N]$ | Główna siła osiowa (ciężar). | Ściskanie + Zginanie (przez mimośród). |
| **L** | $[mm]$ | Długość fizyczna słupa. | Stateczność (wyboczenie), Ugięcia ($L^3$). |
| **Ramię** | $[mm]$ | Mimośród przyłożenia siły $F_x$. | Generuje główny moment zginający $M_z = F_x \cdot (R - y_c)$. |
| **w_Ty** | $[-]$ | Wsp. siły bocznej $F_y$. | Zginanie względem osi słabej. Np. `0.2` to 20% siły $F_x$. |
| **w_Tz** | $[-]$ | Wsp. siły bocznej $F_z$. | **Krytyczny parametr.** Generuje skręcanie i zginanie boczne. |

### B. Współczynniki Bezpieczeństwa (Eurokod)
Domyślne wartości są zgodne z PN-EN 1993-1-1 dla profili spawanych.
* **Gamma M0 (1.0):** Nośność przekroju.
* **Gamma M1 (1.0):** Stateczność elementu.
* **Alfa Imp (0.49):** Krzywa wyboczeniowa 'c' (dla ceowników i profili spawanych).

### C. Tryb Pracy (Mode)
Program oferuje dwa tryby działania:

1.  **🤖 AUTO (Zalecane):**
    * Algorytm sam przeszukuje katalog hutniczy.
    * **Algorytm:** *Standard V3.0 (Smart Step)*. Zaczyna od najlżejszych profili i szuka najcieńszego płaskownika spełniającego warunki.
    * **Filtry:** `Min Otwarcie` (np. dla wózka widłowego) i `Max Grub` (ekonomika).

2.  **📐 MANUAL:**
    * Służy do szybkiego sprawdzenia konkretnego profilu (np. "Co się stanie, jak dam UPE240 + blacha 12?").
    * Wynik pojawia się natychmiast w konsoli.

> **💡 Pro Tip:** W trybie AUTO możesz wybrać **kilka materiałów** naraz (np. Stal S355 i Aluminium AW-6060). Program przeliczy je równolegle.

## 3. KROK 2: Analiza i Selekcja (Zakładka 3: Selektor)

Po zakończeniu obliczeń (przycisk **URUCHOM**), wyniki trafiają tutaj. Tabela może zawierać tysiące wariantów.

### Zarządzanie Tabelą
* **Sortowanie:** Kliknij nagłówek kolumny (np. `Waga`), aby znaleźć najlżejsze rozwiązanie.
* **Status Wymogów:** Kolumna ta pokazuje `SPEŁNIA` (Zielony) lub `NIE SPEŁNIA` (Czerwony). 
    * *Uwaga:* Domyślnie program pokazuje też wyniki niespełniające warunków (dla celów badawczych). Użyj filtra, aby je ukryć.

### Panel Filtrów (Advanced Filtering)
Znajduje się po lewej stronie. Pozwala zawęzić zbiór wyników.
1.  Kliknij `+ Filtr`.
2.  Wybierz kolumnę, np. `Res_UR` (Wytężenie).
3.  Ustaw Max: `1.0`.
4.  Kliknij `Zastosuj` (lub zaznacz checkbox `Pokaż Ukryte` aby odświeżyć).

### Selekcja do FEM
To najważniejszy moment. Wybierasz "zwycięzców" etapu analitycznego do weryfikacji numerycznej.
1.  Zaznacz checkbox w pierwszej kolumnie **PRZEKAZ**.
2.  Możesz zaznaczyć wiele wierszy (np. 3 najlżejsze profile).
3.  Kliknij zielony przycisk **PRZEKAŻ DO FEM ➡️**.

## 4. KROK 3: Weryfikacja Numeryczna (Zakładka 4: Analiza FEM)

W tej zakładce uruchamiany jest solver **CalculiX** (ccx). Program automatycznie generuje geometrię (`.step`, `.geo`), siatkę (`.msh`) i pliki sterujące (`.inp`).

### Konfiguracja Solvera
Dla większości przypadków domyślne ustawienia są optymalne.

| Parametr | Domyślnie | Opis |
| :--- | :--- | :--- |
| **Startowy rozmiar siatki** | 15.0 mm | Bazowa wielkość elementu. Program automatycznie ją zmniejszy, jeśli natrafi na cienką ściankę (np. środnik 6mm). |
| **Wsp. zagęszczania** | 0.7 | W każdej iteracji siatka jest mnożona przez ten czynnik (15 -> 10.5 -> 7.35). |
| **Max Iteracji** | 3 | Liczba pętli $h$-refinementu. Zazwyczaj zbieżność osiągana jest w 2-3 kroku. |
| **Strefy Zagęszczania** | WEBS/FLANGES | Możesz zdefiniować obszary, gdzie siatka ma być gęstsza (np. spoiny). |

### Tryby Uruchomienia
* **URUCHOM PILOTA:** Liczy tylko pierwszy profil z listy. Służy do szybkiego sprawdzenia, czy model nie "wybucha" i czy obciążenia są przyłożone w dobrą stronę.
* **URUCHOM PEŁNY BATCH:** Przetwarza kolejkę wszystkich profili. Wyniki zapisywane są na bieżąco.

### Wizualizacja 3D
Po obliczeniach kliknij **👁️ WIZUALIZACJA 3D**. Otworzy się okno interaktywne:
* **Lewy przycisk:** Obrót.
* **Środkowy przycisk:** Przesuwanie.
* **Rolka:** Przybliżanie.
* **Drzewo (Prawa strona):** Możesz włączać/wyłączać warstwy (Siatka, Wyniki, Grupy).

## 5. KROK 4: Raport i Decyzja (Zakładka 5: Porównanie)

Moduł ten służy do ostatecznej walidacji modelu. Porównuje wyniki szybkie (Analityka) z dokładnymi (FEM).

### Tabela Zgodności (Delta)
Kluczowa jest kolumna **Delta [%]**.

| Kolor Statusu | Zakres Delty | Interpretacja Inżynierska |
| :--- | :--- | :--- |
| <span style="color:green">**ZIELONY**</span> | $< 10\%$ | **Model Poprawny.** Wyniki są zbieżne. Można bezpiecznie projektować. |
| <span style="color:orange">**ŻÓŁTY**</span> | $10 - 20\%$ | **Akceptowalne.** Różnice mogą wynikać z uproszczeń teorii belkowej (np. w narożach). |
| <span style="color:red">**CZERWONY**</span> | $> 20\%$ | **Wymaga Uwagi.** Sprawdź warunki brzegowe w FEM lub czy analityka nie pomija jakiegoś zjawiska (np. lokalnego wyboczenia). |

### Wykresy Ugięć
Program rysuje linię ugięcia belki $u(x)$.
* **Linia przerywana (Niebieska):** Teoretyczna linia ugięcia wg wzorów wytrzymałościowych.
* **Punkty/Linia (Czerwona):** Rzeczywiste przemieszczenia węzłów z symulacji FEM.
* **Wniosek:** Jeśli linie się pokrywają, sztywność modelu jest zdefiniowana poprawnie.

## 6. Rozwiązywanie Problemów (FAQ)

**Q: Wynik FEM (Max Stress) jest znacznie niższy niż analityczny (np. 95 vs 300 MPa).**
A: To typowe dla zgrubnej siatki. CalculiX w pliku `.dat` podaje naprężenia w punktach całkowania (wewnątrz materiału). Przy cienkiej ściance i jednym elemencie na grubość, punkt ten leży blisko osi obojętnej.
*Rozwiązanie:* Zaufaj procesowi zagęszczania (Refinement). W 2. i 3. iteracji wynik wzrośnie i zbliży się do analitycznego. Sprawdź też mapę 3D – tam kolory pokazują wartości na powierzchni.

**Q: Błąd "Mesh Generation Failed".**
A: Prawdopodobnie zadany rozmiar siatki jest większy niż grubość ścianki, co uniemożliwia utworzenie poprawnej geometrii.
*Rozwiązanie:* Program w wersji 4.0 posiada **Auto-Check**. Powinien sam wykryć ten problem i zmniejszyć siatkę. Jeśli błąd nadal występuje, zmniejsz ręcznie "Startowy rozmiar siatki" w Tab 4.

**Q: Program nie widzi `ccx`.**
A: Upewnij się, że plik `ccx.exe` znajduje się w folderze `solver_bin` w katalogu głównym projektu lub jest dodany do zmiennej środowiskowej PATH.

In [ ]:
# PRZYKŁAD URUCHOMIENIA Z KONSOLI (Dla zaawansowanych)
# Jeśli nie chcesz używać GUI, możesz uruchomić program bezpośrednio:

# python app_gui.py

# Lub uruchomić sam solver analityczny dla testów:
# python solvers_opt/solver_1_standard.py